In [44]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import random
import math
from sklearn.metrics import average_precision_score
from sklearn.preprocessing import label_binarize

% matplotlib inline

In [2]:
dtype={'is_booking':bool,
        'srch_ci' : np.str_,
        'srch_co' : np.str_,
        'srch_adults_cnt' : np.int32,
        'srch_children_cnt' : np.int32,
        'srch_rm_cnt' : np.int32,
        'srch_destination_id':np.int32,
        'user_location_country' : np.float64,
        'user_location_region' : np.float64,
        'user_location_city' : np.float64,
        'hotel_cluster' : np.float64,
        'orig_destination_distance':np.float64,
        'date_time':np.str_,
        'hotel_market':np.int32}

In [3]:
df0 = pd.read_csv('train.csv',dtype=dtype, usecols=dtype, parse_dates=['date_time'] ,sep=',',nrows=2500000).dropna()

In [4]:
df2 = df0[(df0['hotel_cluster'] == 91)  | (df0['hotel_cluster'] == 41)| (df0['hotel_cluster'] == 65) | (df0['hotel_cluster'] == 48) | (df0['hotel_cluster'] == 25)| (df0['hotel_cluster'] == 33)| (df0['hotel_cluster'] == 95)| (df0['hotel_cluster'] == 18)| (df0['hotel_cluster'] == 21) | (df0['hotel_cluster'] == 70) ]
#| (df0['hotel_cluster'] == 65) | (df0['hotel_cluster'] == 48) | (df0['hotel_cluster'] == 25)

In [5]:
df1 = df2[[
    'date_time',
    'user_location_country',
    'orig_destination_distance',
    'srch_ci',
    'srch_co', 
    'srch_adults_cnt',
    'srch_children_cnt',
    'srch_rm_cnt', 
    'srch_destination_id',
    'is_booking',
    'hotel_market', 
    'hotel_cluster',
    ]]

In [6]:
df1['srch_ci']=pd.to_datetime(df1['srch_ci'],infer_datetime_format = True,errors='coerce')
df1['srch_co']=pd.to_datetime(df1['srch_co'],infer_datetime_format = True,errors='coerce')
df1['date_time']=pd.to_datetime(df1['date_time'],infer_datetime_format = True,errors='coerce')

df1['month']= df1['date_time'].dt.month
df1['plan_time'] = ((df1['srch_ci']-df1['date_time'])/np.timedelta64(1,'D')).astype(float)
df1['length_of_stay']=((df1['srch_co']-df1['srch_ci'])/np.timedelta64(1,'D')).astype(float)

/Users/abhisheknigam/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/abhisheknigam/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/abhisheknigam/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value

In [7]:
df1.loc[df1['srch_children_cnt'] > 0, 'srch_children_cnt'] = 1
df1.loc[df1['srch_adults_cnt'] > 2, 'srch_adults_cnt'] = 3

In [8]:
#['is_booking','hotel_market','user_location_country','user_location_region','user_location_city','srch_destination_id']
#['date_time','srch_ci','srch_co']
lst_drop=['date_time','srch_ci','srch_co','is_booking','hotel_market','user_location_country','srch_destination_id']
df1.drop(lst_drop,axis=1,inplace=True)

In [9]:
msk = np.random.rand(len(df1)) < 0.8

In [10]:
train = df1[msk]

In [11]:
test = df1[~msk]

In [12]:
expected_result = test.hotel_cluster

In [13]:
del test['hotel_cluster']

In [14]:
del df1
del df2

In [15]:
trainingSet = train.as_matrix()
testSet = test.as_matrix()

# Naive Bayes Implementation

In [16]:
def segregateClass(df):
    classLabel = {}
    for i in range(len(df)):
        array = df[i]
        if (array[-4] not in classLabel):
            classLabel[array[-4]] = []
        classLabel[array[-4]].append(array)
    return classLabel

In [17]:
def calculateStandardDeviation(arrayOfNum):
    avg = calculateMean(arrayOfNum)
    variance = sum([pow(x-avg,2) for x in arrayOfNum])/float(len(arrayOfNum)-1)
    return math.sqrt(variance)

In [18]:
def calculateMean(arrayOfNum):
    return sum(arrayOfNum)/float(len(arrayOfNum))

In [19]:
def segregateClasses(dataset):
    classSummary = [(calculateMean(attribute), calculateStandardDeviation(attribute)) for attribute in zip(*dataset)]
    del classSummary[-4]
    return classSummary

In [20]:
def summarizeByClass(dataset):
    differentClasses = segregateClass(dataset)
    classSummaries = {}
    for value, instances in differentClasses.items():
        classSummaries[value] = segregateClasses(instances)
    return classSummaries

In [21]:
import math
def calculateProbability(x, mean, stdev):
    exponent = math.exp(-(math.pow(x-mean,2)/(2*math.pow(stdev,2))))
    return (1 / (math.sqrt(2*math.pi) * stdev)) * exponent

In [22]:
def probabilityForEachClass(summaries, inputVector):
    probabilities = {}
    for classValue, classSummaries in summaries.items():
        probabilities[classValue] = 1
        for i in range(len(classSummaries)):
            mean, stdev = classSummaries[i]
            x = inputVector[i]
            probabilities[classValue] += calculateProbability(x, mean, stdev)
    return probabilities

In [23]:
def getClassPrediction(summaries, inputVector):
    probabilities = probabilityForEachClass(summaries, inputVector)
    bestLabel, bestProb = None, -1
    for classValue, probability in probabilities.items():
        if bestLabel is None or probability > bestProb:
            bestProb = probability
            bestLabel = classValue
    return bestLabel

In [24]:
def predict(summaries, testSet):
	predictions = []
	for i in range(len(testSet)):
		result = getClassPrediction(summaries, testSet[i])
		predictions.append(result)
	return predictions

In [25]:
def accuracy(actual_values, predicted_values):
    match = 0
    for x in range(len(actual_values)):
        if actual_values['hotel_cluster'][x] == predicted_values[x]:
            match += 1
    return (match/float(len(actual_values))) * 100.0

# Running Naive Bayes on the dataset

In [26]:
summaries = summarizeByClass(trainingSet)

In [27]:
predicted_value = predict(summaries, testSet)

In [28]:
pd_final = pd.DataFrame(data=expected_result, dtype=np.int64)

In [29]:
pd_final = pd_final.reset_index()

In [30]:
accuracy = accuracy(pd_final, predicted_value)

In [31]:
accuracy

15.884505896705978

In [48]:
y = label_binarize(pd_final['hotel_cluster'], classes=[91,41,65,48,25,33,95,18,21,70])
predicted_value = label_binarize(predicted_value, classes=[91,41,65,48,25,33,95,18,21,70])

In [49]:
average_precision = average_precision_score(y, predicted_value)
print('Average precision-recall score: {0:0.2f}'.format(
      average_precision))

Average precision-recall score: 0.24


In [ ]:
pd_final['hotel_cluster'][1]

In [ ]:
predicted_value[1]

In [40]:
pd_final['hotel_cluster']

0        25
1        95
2        95
3        95
4        95
5        95
6        95
7        95
8        91
9        25
10       25
11       25
12       25
13       70
14       95
15       65
16       65
17       33
18       33
19       33
20       33
21       33
22       33
23       33
24       33
25       33
26       33
27       33
28       33
29       91
         ..
61445    21
61446    95
61447    18
61448    95
61449    21
61450    18
61451    21
61452    70
61453    70
61454    91
61455    48
61456    41
61457    41
61458    91
61459    91
61460    18
61461    18
61462    25
61463    91
61464    25
61465    41
61466    41
61467    65
61468    70
61469    91
61470    48
61471    91
61472    41
61473    91
61474    41
Name: hotel_cluster, dtype: int64